Net stuff

In [1]:
import httpx
import asyncio
from fredapi import Fred
import requests
import numpy as np
import json
import pandas as pd
import pickle
import yahoo_fin.stock_info as si
# We use normal datetime for fred info and pandas datetime for data
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import islice
import math
import os
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.stats import hmean
from itertools import product
import sys
import cProfile
import yfinance as yf
import warnings
from collections import defaultdict

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [70]:
modules_to_reload = ["extract_functions", "reshape", "conversions", "net_stuff", "cleanup"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

In [ ]:
#NetCashProvidedByUsedInOperatingActivities NetCashProvidedByUsedInOperatingActivitiesContinuingOperations"WeightedAverageNumberOfSharesOutstandingBasic", "WeightedAverageNumberOfDilutedSharesOutstanding", EntityCommonStockSharesOutstanding
#"static":["Assets", "Liabilities", "AssetsCurrent", "AssetsNoncurrent"],"dynamic":["FreeCashFlow","GrossProfit"]
comp = comp_load("MSFT")
paths = recursive_date_gather(comp,"AccountsReceivableNet", "static")
print(paths)
frame, unit = path_selector(comp,"AccountsReceivableNet",paths["paths"][0], True, dynamic_tolerance=pd.Timedelta(days=1000), static_tolerance=pd.Timedelta(days=1000), reshape_approx = False, forced_dynamic=True)
# frame.to_csv("..\examples/HardearneCOGS9.csv")
print(frame)
# frame.to_csv("..\examples\\realAAPLgross.csv")


In [61]:
comp = comp_load("MSFT")
for i,datapoint in enumerate(list(comp.data['AccountsReceivableNet']["units"].items())[0][1]):
    print(datapoint)

{'end': '2016-07-01', 'val': 22300000000, 'accn': '0001564590-18-019062', 'fy': 2018, 'fp': 'FY', 'form': '10-K', 'filed': '2018-08-03', 'frame': 'CY2016Q2I'}


In [ ]:
company_wordsearch("GOOGL", "short")

Datapoint reshape

In [ ]:
comp = comp_load("GS")
data= comp.converted_data["StockholdersEquity"]
reshaped, intervals, dynamic = reshape('StockholdersEquity', data, comp.ticker, converted = True, use_precompute=False, approx=True, forced_dynamic=True)
print(intervals)
for key,value in list(reshaped.items())[:]:
        print(key,value[0])

Functions

In [4]:
FACTS_PATH = r"C:\Edgar_data"
SUBMISSIONS_PATH = r"C:\Submissions_data"

#Unavailable stuff
with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

#Lookup table for the undeprecated version of a measure
with open(r"..\other_pickle\deprecated_to_current.json", "r") as file:
    deprecate_conversion = json.load(file)
    file.close()

#Categories
with open(r"..\categories\categories.json", "r") as file:
    categories = json.load(file)

#Irrelevants
with open(r"..\categories\category_measures_irrelevant.json") as file:
    irrelevants = json.load(file)

#Make sure the necessary folders exist
for category, num_range in categories.items():
    os.makedirs(f"..\\companies_data\static\{category}", exist_ok=True)
    os.makedirs(f"..\\companies_data\dynamic\{category}", exist_ok=True)
    os.makedirs(f"..\\clean_data\static\{category}", exist_ok=True)
    os.makedirs(f"..\\clean_data\dynamic\{category}", exist_ok=True)
    os.makedirs(f"..\\companies_data_missing\{category}", exist_ok=True)
    os.makedirs(f"..\clean_data\per_share\static\{category}", exist_ok=True)
    os.makedirs(f"..\clean_data\per_share\dynamic\{category}", exist_ok=True)
    
os.makedirs(f"..\\companies_data\price", exist_ok=True)
os.makedirs(f"..\\clean_data\price", exist_ok=True)
os.makedirs(f"..\\ready_data\static", exist_ok=True)
os.makedirs(f"..\\ready_data\dynamic", exist_ok=True)
os.makedirs(f"..\\ready_data\\indicator", exist_ok=True)
os.makedirs(f"..\\ready_data\\together", exist_ok=True)

with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

for ticker, available in company_frames_availability.items():
    os.makedirs(f"..\\reshaped\{ticker}", exist_ok=True)

for path in ["checkout", "companies", "companies_data", "companies_data_missing", "units-checkout"]:
    os.makedirs("..\\" + path, exist_ok=True)

with open("..\categories\category_measures.json", "r") as file:
    category_measures = json.load(file)

#Get the measures that you will attempt to get 
with open("..\other_pickle\measures.json", "r") as file:
    Measures = json.load(file)

#The first entry date into the EDGAR database
START = datetime.strptime('1993-01-01', r"%Y-%m-%d")

#Headers for EDGAR call
headers = {
    "User-Agent":"ficakc@seznam.cz",
    "Accept-Encoding":"gzip, deflate",
}

# cik_url =  "https://www.sec.gov/files/company_tickers.json"
# cikdata = requests.get(cik_url, headers=headers).json()

with open(r"..\other_pickle\cik.json","r") as file:
    cikdata = json.load(file)

months_dict = {1: 90, 2: 92, 3: 91, 4: 92, 5: 92, 6: 92, 7: 92, 8: 91, 9: 92, 10: 92, 11: 90, 12: 90}

Indicator frame


In [15]:
indicators = ["TB3MS", "DCOILWTICO","CORESTICKM159SFRBATL"]
indicator_frame = fred_info(indicators, START, datetime.now().date())
indicator_frame.to_csv('..\other_pickle\indicator_frame.csv')

Data grab

In [68]:
#write out measures based on importance in descending order
with open("..\other_pickle\measures.json", "r") as file:
    fund_measures = json.load(file)
#Load out the indicators 
with open("..\other_pickle\indicator_frame.pkl", "rb") as file:
    indicator_frame = pickle.load(file)

#Load the info about the companies that we already have (ticker,edgar,yahoo)
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

#Load the unavailable data so far 
with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

edgar_client =  httpx.AsyncClient()
sem_edgar = asyncio.Semaphore(9)
#Separate sem for yahoo to spread the work and connections
sem_yahoo = asyncio.Semaphore(30)

#Create tasks to ge the first companies_num companies by valuation
# company_frames_availability = {"LLY":((0,0),0)}
companies_num = 1000
ticker_dict = dict(islice(company_frames_availability.items(), companies_num))
ticker_list = ticker_fill(ticker_dict)
tasks = []
for ticker in ticker_list:
    tasks.append(async_task(ticker, edgar_client, sem_edgar, sem_yahoo, fund_measures))

availability_list = await asyncio.gather(*tasks)

for ping in availability_list:
    try:
        ticker, avail = ping
        company_frames_availability[ticker] = avail
        if avail == "del":
            company_frames_availability[ticker] = ["del", "del"]
    except Exception as e:
         company_frames_availability[ticker] = ["del", "del"]

success_rate(availability_list)

print(company_frames_availability)
with open(r"..\other_pickle\frame_availability.pkl", "wb") as file:
    pickle.dump(company_frames_availability, file)

#Save the unavailable measures for later use
with open(r'..\other_pickle\unavailable.json', 'w') as file:
        json.dump(Unavailable_Measures,file, indent= 1)

Loading META
META:43.76727375806109%
Price pinging META$
Loading JPM
JPM:47.49156411274315%
Price pinging JPM$
Loading UNH
UNH:46.321975141625096%
Price pinging UNH$
Loading COST
COST:48.7116505321967%
Price pinging COST$
Loading CVX
CVX:48.56726379991561%
Price pinging CVX$
Loading MRK
MRK:43.25507026845073%
Price pinging MRK$
Loading PEP
PEP:51.13230035756854%
Price pinging PEP$
Loading AZN
AZN:55.85106382978723%
Price pinging AZN$
Loading TMUS
TMUS:47.02742349537509%
Price pinging TMUS$
Loading HDB
HDB:17.30991449040906%
Price pinging HDB$
Loading BX
BX:45.953324055240344%
Price pinging BX$
Loading GE
GE:40.227000846147114%
Price pinging GE$
Loading AXP
AXP:44.366993217784476%
Price pinging AXP$
Loading T
T:45.43933054393305%
Price pinging T$
Loading ISRG
No USD like unit available for ISRG : dict_keys(['classActionComplaint'])
ISRG:44.918389553862895%
Price pinging ISRG$
Loading SONY
SONY:52.554416261917616%
No USD like unit available for SONY : dict_keys(['JPY'])
Loading TD
TD:35.

TypeError: cannot unpack non-iterable NoneType object

In [65]:
print(company_frames_availability)
# with open(r"..\other_pickle\frame_availability.pkl", "wb") as file:
#     pickle.dump(company_frames_availability, file)

||Done BR||
{'AAPL': [(1, 1), 1], 'MSFT': [(1, 1), 1], 'GOOGL': [(1, 1), 1], 'AMZN': [(1, 1), 1], 'NVDA': [(1, 1), 1], 'META': [(0, 1), 1], 'TSLA': [(1, 1), 1], 'BRK-B': [(1, 1), 1], 'LLY': [(1, 1), 1], 'TSM': ['del', 'del'], 'V': [(1, 1), 1], 'AVGO': [(1, 1), 1], 'JPM': [(0, 1), 1], 'UNH': [(0, 1), 1], 'NVO': ['del', 'del'], 'SPY': ['del', 'del'], 'WMT': [(1, 1), 1], 'LVMUY': ['del', 'del'], 'XOM': [(1, 1), 1], 'MA': [(1, 1), 1], 'JNJ': [(1, 1), 1], 'LTMAY': ['del', 'del'], 'HD': [(1, 1), 1], 'PG': [(1, 1), 1], 'ASML': ['del', 'del'], 'COST': [(1, 0), 1], 'ORCL': [(1, 1), 1], 'CVX': [(0, 1), 1], 'MRK': [(0, 1), 1], 'ABBV': [(1, 1), 1], 'ADBE': [(1, 1), 1], 'BAC': [(1, 1), 1], 'CRM': [(1, 1), 1], 'KO': [(1, 1), 1], 'TM': ['del', 'del'], 'AMD': [(1, 1), 1], 'FMX': ['del', 'del'], 'PEP': [(0, 1), 1], 'ACN': [(1, 1), 1], 'NFLX': [(1, 1), 1], 'MCD': [(1, 1), 1], 'SHEL': [(1, 1), 1], 'INTC': [(1, 1), 1], 'AZN': [(0, 1), 1], 'TMO': [(1, 1), 1], 'NVS': [(1, 1), 1], 'CSCO': [(1, 1), 1], 'LIN':

||Done SYM||
||Done STT||
||Done ULTA||
||Done FITB||
Yahoo connection error.
||Done EQR||
||Done RJF||
||Done ICLR||
||Done TSCO||
||Done SE||
||Done RIVN||
||Done CHD||
||Done MTB||
||Done WBA||
||Done WAB||
||Done ILMN||
||Done WPM||
||Done DTE||
||Done PHM||
||Done EBAY||
||Done HWM||
||Done ZM||
||Done HPE||
Yahoo error:{'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
||Done ARE||
||Done CUK||
||Done LYV||
||Done AWK||
||Done STE||
||Done FLT||
||Done ES||
||Done FNV||
||Done DOV||
||Done EXPE||
||Done ETR||
||Done OWL||
||Done SIRI||
||Done INVH||
||Done ALGN||
||Done ROL||
||Done PTC||
||Done TDY||
||Done VRSN||
||Done BLDR||
||Done IFF||
||Done MOH||
||Done IRM||
||Done JBHT||
||Done VTR||
||Done BRO||
||Done KOF||
||Done PPL||
||Done DRI||
||Done WAT||
||Done AXON||
||Done BGNE||
||Done BAX||
||Done CTRA||
||Done GPC||
||Done TRGP||
||Done TSN||
||Done LH||
||Done AEE||
||Done COO||
||Done CBOE||
||Done NVR||
|

Resets

In [44]:
#BIG RESET!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#GATHER THE FIRST companies_num companies ciks and pass them to the gather with the tasks
company_frames_availability = {}
for company, values in cikdata.items():
    company_frames_availability[values["ticker"]]=[(0,0),0]
print(company_frames_availability)
with open(r"..\other_pickle\frame_availability.pkl", "wb") as file:
    pickle.dump(company_frames_availability, file)


{'AAPL': [(0, 0), 0], 'MSFT': [(0, 0), 0], 'GOOGL': [(0, 0), 0], 'AMZN': [(0, 0), 0], 'NVDA': [(0, 0), 0], 'META': [(0, 0), 0], 'TSLA': [(0, 0), 0], 'BRK-B': [(0, 0), 0], 'LLY': [(0, 0), 0], 'TSM': [(0, 0), 0], 'V': [(0, 0), 0], 'AVGO': [(0, 0), 0], 'JPM': [(0, 0), 0], 'UNH': [(0, 0), 0], 'NVO': [(0, 0), 0], 'SPY': [(0, 0), 0], 'WMT': [(0, 0), 0], 'LVMUY': [(0, 0), 0], 'XOM': [(0, 0), 0], 'MA': [(0, 0), 0], 'JNJ': [(0, 0), 0], 'LTMAY': [(0, 0), 0], 'HD': [(0, 0), 0], 'PG': [(0, 0), 0], 'ASML': [(0, 0), 0], 'COST': [(0, 0), 0], 'ORCL': [(0, 0), 0], 'CVX': [(0, 0), 0], 'MRK': [(0, 0), 0], 'ABBV': [(0, 0), 0], 'ADBE': [(0, 0), 0], 'BAC': [(0, 0), 0], 'CRM': [(0, 0), 0], 'KO': [(0, 0), 0], 'TM': [(0, 0), 0], 'AMD': [(0, 0), 0], 'FMX': [(0, 0), 0], 'PEP': [(0, 0), 0], 'ACN': [(0, 0), 0], 'NFLX': [(0, 0), 0], 'MCD': [(0, 0), 0], 'SHEL': [(0, 0), 0], 'INTC': [(0, 0), 0], 'AZN': [(0, 0), 0], 'TMO': [(0, 0), 0], 'NVS': [(0, 0), 0], 'CSCO': [(0, 0), 0], 'LIN': [(0, 0), 0], 'BABA': [(0, 0), 0], '

In [16]:
#Frames loaded reset
company_frames_computed = {}
for company, values in cikdata.items():
    if company_frames_availability[values["ticker"]][0] == "del":
        continue
    else:
        company_frames_computed[values["ticker"]]=[0,0,0]
print(company_frames_computed)
with open(r"..\other_pickle\frames_computed_availability.pkl", "wb") as file:
    pickle.dump(company_frames_computed, file)

{'AAPL': [0, 0, 0], 'MSFT': [0, 0, 0], 'GOOGL': [0, 0, 0], 'AMZN': [0, 0, 0], 'NVDA': [0, 0, 0], 'META': [0, 0, 0], 'TSLA': [0, 0, 0], 'BRK-B': [0, 0, 0], 'LLY': [0, 0, 0], 'V': [0, 0, 0], 'AVGO': [0, 0, 0], 'JPM': [0, 0, 0], 'UNH': [0, 0, 0], 'WMT': [0, 0, 0], 'XOM': [0, 0, 0], 'MA': [0, 0, 0], 'JNJ': [0, 0, 0], 'HD': [0, 0, 0], 'PG': [0, 0, 0], 'COST': [0, 0, 0], 'ORCL': [0, 0, 0], 'CVX': [0, 0, 0], 'MRK': [0, 0, 0], 'ABBV': [0, 0, 0], 'ADBE': [0, 0, 0], 'BAC': [0, 0, 0], 'CRM': [0, 0, 0], 'KO': [0, 0, 0], 'AMD': [0, 0, 0], 'PEP': [0, 0, 0], 'ACN': [0, 0, 0], 'NFLX': [0, 0, 0], 'MCD': [0, 0, 0], 'SHEL': [0, 0, 0], 'INTC': [0, 0, 0], 'AZN': [0, 0, 0], 'TMO': [0, 0, 0], 'NVS': [0, 0, 0], 'CSCO': [0, 0, 0], 'LIN': [0, 0, 0], 'PDD': [0, 0, 0], 'ABT': [0, 0, 0], 'TMUS': [0, 0, 0], 'WFC': [0, 0, 0], 'CMCSA': [0, 0, 0], 'INTU': [0, 0, 0], 'DHR': [0, 0, 0], 'HDB': [0, 0, 0], 'NKE': [0, 0, 0], 'DIS': [0, 0, 0], 'QCOM': [0, 0, 0], 'PFE': [0, 0, 0], 'BX': [0, 0, 0], 'TTE': [0, 0, 0], 'VZ': [0, 

In [92]:
#UNAVAILABLE RESET
with open(r"..\categories\categories.json", "r") as file:
      categories = json.load(file)
Unavailable_Measures = {}
for category, num_range in categories.items():
    Unavailable_Measures[category] = {}
with open(r'..\other_pickle\unavailable.json', 'w') as file:
        json.dump(Unavailable_Measures,file, indent = 1)

Missing compile

In [93]:
healthy = 0
total = 0
infected = []
for ticker, availability in company_frames_availability.items():
    if availability[0] == "del":
        continue
    (static, dynamic) = availability[0]
    yahoo = availability[1]
    if (static or dynamic) and yahoo == True:
        total += 1
        comp = comp_load(ticker)
        # print(comp.ticker)
        if comp.missing['static'] != [] or comp.missing['dynamic'] != []:
            catandticker =  comp.ticker +" - "+ comp.sic_desc
            cat = get_category(comp.sic)
            Unavailable_Measures[cat][catandticker] = comp.missing['static']
            Unavailable_Measures[cat][catandticker] += comp.missing['dynamic']
            infected.append(comp.ticker)
        else:
            healthy += 1

print(f"{100*healthy/total}%")
    
with open(r'..\other_pickle\unavailable.json', 'w') as file:
    json.dump(Unavailable_Measures,file, indent= 1)



53.28947368421053%


In [68]:
for module in ["extract_functions", "reshape", "conversions", "net_stuff","cleanup"]:
    if module in sys.modules:
        del sys.modules[module]
    import extract_functions
    from extract_functions import *

Get the frames 

In [18]:
with open("..\other_pickle\indicator_frame.pkl", "rb") as file:
    indicator_frame = pickle.load(file)

with open(r"..\other_pickle\frames_computed_availability.pkl", "rb") as file:
    company_frames_computed_availability = pickle.load(file)

#Load the info about the companies that we already have (ticker,edgar,yahoo)
with open(r"..\other_pickle\frame_availability.pkl", "rb") as file:
    company_frames_availability = pickle.load(file)

with open(r'..\other_pickle\unavailable.json', 'r') as file:
        Unavailable_Measures = json.load(file)

with open(r"..\other_pickle\measures.json", "r") as file:
    measures = json.load(file)

# measures = {"static":["Assets", "Liabilities", "AssetsCurrent", "AssetsNoncurrent", "InventoryNet"],"dynamic":["FreeCashFlow","GrossProfit"]}
# company_frames_availability = {"BA":[(1,1),1]}

computed_iterator = list(company_frames_computed_availability.values())
fuckers = 0
shit_leopards = ["GS"]
stringers = 0
i = 0
for ticker, availability in list(company_frames_availability.items())[:]:
    i += 1
    if availability[0] == "del":
        continue 
    (static, dynamic), yahoo = availability
    if (static or dynamic) and yahoo:
        if not computed_iterator[i][0] or not computed_iterator[i][1] or not computed_iterator[i][2]: #Check if we don't have the frames already loaded
            comp = comp_load(ticker)
            catg = get_category(comp.sic)
            print(f"Getting {comp.ticker} in {catg}")
            try:
                #frames is a dict with static and dynamic frames, if static or dynamic is inelligible, it is not in the dict 
                frames = acquire_frame(comp, measures,(static, dynamic), indicator_frame, reshape_approx=True, lookbehind =4, dynamic_tolerance=pd.Timedelta(days=1000), static_tolerance=pd.Timedelta(days=400), forced_dynamic=True) #category_measures[catg]
                if not frames["static"].empty :
                    if static and comp.missing["static"] == []:
                        frames["static"].to_csv(f"..\companies_data\static\{catg}\{ticker}.csv")
                        company_frames_computed_availability[ticker][0] = 1
                if not frames["dynamic"].empty:
                    if dynamic and comp.missing["dynamic"] == []:
                        frames["dynamic"].to_csv(f"..\companies_data\dynamic\{catg}\{ticker}.csv")
                        company_frames_computed_availability[ticker][1] = 1
                #We also get the aditional columns 
                price_frame = get_label_columns(comp)
                if not price_frame.empty:
                    price_frame.to_csv(f"..\companies_data\price\{ticker}.csv")
                    company_frames_computed_availability[ticker][2] = 1
                # Save it again because of time_init and also save some memory
                with open(f'..\companies\{ticker}.pkl', 'wb') as file:
                    pickle.dump(comp,file)
                    del comp
                print(f"{ticker} saved.")  
            except TypeError as e:
                print(f"{ticker} was supposed to be deleted")
                stringers += 1
            except Exception as e:
                if ticker in shit_leopards:
                    print("Shit leopard")
                else:
                    print(f"{ticker} is fucked in the head: {e}")
                    fuckers += 1    
                company_frames_computed_availability[ticker] = "del"
with open(r"..\other_pickle\frames_computed_availability.pkl", "wb") as file:
    pickle.dump(company_frames_computed_availability, file)
print(fuckers, stringers)

Getting AAPL in Manufacturing, Technology and Heavy Machinery
AAPL Getting AccountsPayableCurrent
{'concat': [({'AccountsPayableCurrent': True}, [(Timestamp('2009-12-26 00:00:00'), Timestamp('2023-12-30 00:00:00'))])]}
AAPL Getting AccountsReceivableNet
{'AccountsReceivableNet': {'add': {'AccountsReceivableNetCurrent': True}}}
AAPL Getting Assets
{'concat': [({'Assets': True}, [(Timestamp('2009-09-26 00:00:00'), Timestamp('2023-12-30 00:00:00'))])]}
AAPL Getting AssetsCurrent
{'AssetsCurrent': True}
AAPL Getting CashAndCashEquivalentsAtCarryingValue
{'CashAndCashEquivalentsAtCarryingValue': True}
AAPL Getting InventoryNet
{'InventoryNet': True}
AAPL Getting Liabilities
{'concat': [({'Liabilities': True}, [(Timestamp('2009-09-26 00:00:00'), Timestamp('2023-12-30 00:00:00'))])]}
AAPL Getting LiabilitiesCurrent
{'LiabilitiesCurrent': True}
AAPL Getting LongTermDebt
{'concat': [({'LongTermDebt': True}, [(Timestamp('2013-09-28 00:00:00'), Timestamp('2015-03-28 00:00:00'))]), ({'LongTermDebt

XBRL 

In [ ]:
Elementframe = pd.read_csv("ListOfElements.csv")
# Elementframe = Elementframe[Elementframe["approvalStatus"].apply(lambda rem: rem=="Final")]
# Elementframe.to_csv("ListOfElements.csv")
def xbrl_wordsearch(word, length):
    matches = Elementframe[Elementframe["elementName"].apply(lambda x: x if (len(x)<length) else "").str.contains(word, na=False)]
    if matches.empty:
        print("No match")
        return
    print(json.dumps(matches[["elementName","definition","Deprecated"]].to_dict(orient="records"), indent = 2))

with open(r"companies\MSFT.pkl", "rb") as file:
    company = pickle.load(file)

with open(r"units.json", "w") as file:
    json.dump(compdict,file, indent= 1)

Splits 

In [186]:
tickerSymbol = 'MSFT'
tickerData = yf.Ticker(tickerSymbol)
tickerDf = tickerData.history(period='1d', start='2010-1-1', end='2020-1-1')
tickerDf.to_csv("..\examples\Otherpricemethod")

In [14]:
comp = comp_load("NVDA")
price_frame = get_label_columns(comp)
price_frame.to_csv("..\examples\\new_price.csv")

In [71]:
modules_to_reload = ["extract_functions", "reshape", "conversions", "net_stuff","cleanup"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

Cleanup

In [72]:
with open(r"..\other_pickle\frames_computed_availability.pkl", "rb") as file:
    company_frames_computed_availability = pickle.load(file)

EOFError: Ran out of input

In [73]:
print(company_frames_computed_availability["GS"])

del


In [74]:
cleanup(company_frames_computed_availability, limit= 100, tether_index=True, ignore_columns=["PropertyPlantAndEquipmentNet"])

Cleaning AAPL
Cleaning MSFT
Cleaning AMZN
Cleaning NVDA
Cleaning META
Cleaning TSLA
Cleaning LLY
Cleaning V
Cleaning AVGO
Cleaning UNH
Cleaning WMT
Cleaning XOM
Cleaning MA
Cleaning JNJ
Cleaning HD
Cleaning PG
Cleaning COST
Cleaning ORCL
Cleaning CVX
Cleaning MRK
Cleaning ABBV
Cleaning ADBE
Cleaning CRM
Cleaning KO
Cleaning AMD
Cleaning PEP
Cleaning ACN
Cleaning NFLX
Cleaning MCD
Cleaning INTC
Cleaning TMO
Cleaning CSCO
Cleaning LIN
Cleaning ABT
Cleaning TMUS
Cleaning CMCSA
Cleaning INTU
Cleaning DHR
Cleaning NKE
Cleaning QCOM
Cleaning PFE
Cleaning BX
Cleaning VZ
Cleaning TXN
Cleaning AMGN
Cleaning CAT
Cleaning UNP
Cleaning IBM
Cleaning PM
Cleaning NOW
Cleaning GE
Cleaning SPGI
Cleaning COP
Cleaning HON
Cleaning AMAT
Cleaning UPS
Cleaning UBER
Cleaning LOW
Cleaning BKNG
Cleaning PLD
Cleaning RTX
Cleaning BLK


In [75]:
ghost(company_frames_computed_availability, limit=100, size = 50, adjustment=0.5, iterations = 2, verbose=True)

Ghost for AAPL
Ghost for MSFT
Ghost for AMZN
Ghost for NVDA
Ghost for META
Ghost for TSLA
Ghost for LLY
Ghost for V
Ghost for AVGO
Ghost for UNH
Ghost for WMT
Ghost for XOM
Ghost for MA
Ghost for JNJ
Ghost for HD
Ghost for PG
Ghost for COST
Ghost for ORCL
Ghost for CVX
Ghost for MRK
Ghost for ABBV
Ghost for ADBE
Ghost for CRM
Ghost for KO
Ghost for AMD
Ghost for PEP
Ghost for ACN
Ghost for NFLX
Ghost for MCD
Ghost for INTC
Ghost for TMO
Ghost for CSCO
Ghost for LIN
Ghost for ABT
Ghost for TMUS
Ghost for CMCSA
Ghost for INTU
Ghost for DHR
Ghost for NKE
Ghost for QCOM
Ghost for PFE
Ghost for BX
Ghost for VZ
Ghost for TXN
Ghost for AMGN
Ghost for CAT
Ghost for UNP
Ghost for IBM
Ghost for PM
Ghost for NOW
Ghost for GE
Ghost for SPGI
Ghost for COP
Ghost for HON
Ghost for AMAT
Ghost for UPS
Ghost for UBER
Ghost for LOW
Ghost for BKNG
Ghost for PLD
Ghost for RTX
Ghost for BLK


In [26]:
quantiles(company_frames_computed_availability, limit=100, quantile = 0.8, period_dict={"static": 30, "dynamic": 90})

Quantile for AAPL
Quantile for MSFT
Quantile for AMZN
Quantile for NVDA
Quantile for META
Quantile for TSLA
Quantile for LLY
Quantile for V
Quantile for AVGO
Quantile for UNH
Quantile for WMT
Quantile for XOM
Quantile for MA
Quantile for JNJ
Quantile for HD
Quantile for PG
Quantile for COST
Quantile for ORCL
Quantile for CVX
Quantile for MRK
Quantile for ABBV
Quantile for ADBE
Quantile for CRM
Quantile for KO
Quantile for AMD
Quantile for PEP
Quantile for ACN
Quantile for NFLX
Quantile for MCD
Quantile for INTC
Quantile for TMO
Quantile for CSCO
Quantile for LIN
Quantile for ABT
Quantile for TMUS
Quantile for CMCSA
Quantile for INTU
Quantile for DHR
Quantile for NKE
Quantile for DIS
Quantile for QCOM
Quantile for PFE
Quantile for BX
Quantile for VZ
Quantile for BA
Quantile for TXN
Quantile for AMGN
Quantile for CAT
Quantile for UNP
Quantile for IBM
Quantile for PM
Quantile for NOW
Quantile for GE
Quantile for SPGI
Quantile for COP
Quantile for HON
Quantile for AMAT
Quantile for UPS
Qua

In [23]:
upper_averages(company_frames_computed_availability, limit=100, quantile = 0.8, period_dict={"static": 30, "dynamic": 90})

Average for AAPL
Average for MSFT
Average for AMZN
Average for NVDA
Average for META
Average for TSLA
Average for LLY
Average for V
Average for AVGO
Average for UNH
Average for WMT
Average for XOM
Average for MA
Average for JNJ
Average for HD
Average for PG
Average for COST
Average for ORCL
Average for CVX
Average for MRK
Average for ABBV
Average for ADBE
Average for CRM
Average for KO
Average for AMD
Average for PEP
Average for ACN
Average for NFLX
Average for MCD
Average for INTC
Average for TMO
Average for CSCO
Average for LIN
Average for ABT
Average for TMUS
Average for CMCSA
Average for INTU
Average for DHR
Average for NKE
Average for DIS
Average for QCOM
Average for PFE
Average for BX
Average for VZ
Average for BA
Average for TXN
Average for AMGN
Average for CAT
Average for UNP
Average for IBM
Average for PM
Average for NOW
Average for GE
Average for SPGI
Average for COP
Average for HON
Average for AMAT
Average for UPS
Average for UBER
Average for LOW
Average for BKNG
Average for

In [ ]:
per_share_divide(company_frames_computed_availability, limit=100)

In [39]:
ready_data(company_frames_computed_availability, limit=100, dynamic_shift= pd.Timedelta(days=120), static_shift =pd.Timedelta(days=90), multiples=True, categories=False, averages = True)

Readying AAPL
Readying MSFT
Readying GOOGL
Readying AMZN
Readying NVDA
Readying META
Readying TSLA
Readying BRK-B
Readying LLY
Readying TSM
Readying V
Readying AVGO
Readying JPM
Readying UNH
Readying NVO
Readying SPY
Readying WMT
Readying LVMUY
Readying XOM
Readying MA
Readying JNJ
Readying LTMAY
Readying HD
Readying PG
Readying ASML
Readying COST
Readying ORCL
Readying CVX
Readying MRK
Readying ABBV
Readying ADBE
Readying BAC
Readying CRM
Readying KO
Readying TM
Readying AMD
Readying FMX
Readying PEP
Readying ACN
Readying NFLX
Readying MCD
Readying SHEL
Readying INTC
Readying AZN
Readying TMO
Readying NVS
Readying CSCO
Readying LIN
Readying BABA
Readying PDD
Readying ABT
Readying TMUS
Readying SAP
Readying WFC
Readying CMCSA
Readying INTU
Readying BHP
Readying DHR
Readying HDB
Readying NKE
Readying DIS
Readying QCOM
Readying PFE
Readying QQQ
Readying BX
Readying TTE
Readying VZ
Readying BA
Readying TXN
Readying HSBC
Readying AMGN
Readying MS
Readying CAT
Readying RTNTF
Readying UNP
Re